In [1]:
from dotenv import load_dotenv

load_dotenv()  # .env 파일을 환경 변수로 로드

True

In [20]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.pipeline import PipelinePromptTemplate
from langchain.prompts import PromptTemplate

In [19]:
chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

In [22]:
intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}
"""
)

example = PromptTemplate.from_template(
    """
    This is an example of how you talk:

    Human: {example_question}
    You: {example_answer}
"""
)

start = PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You:
"""
)

final = PromptTemplate.from_template(
    """
    {intro}

    {example}

    {start}
"""
)

In [26]:
 prompts = [
    ("intro", intro),
    ("example", example),
    ("start", start)
]

 full_prompt = PipelinePromptTemplate(
    final_prompt=final,
    pipeline_prompts=prompts,
)

 chain = full_prompt | chat

 chain.invoke({
     "character" : "Pirate",
     "example_question" : "What is your location?",
     "example_answer" : "Arrrrg! That is a sceret!! Arg arg!",
     "question" : "What is your favorite food?",
 })

Arrrrg! Me favorite food be a good ol' plate of salted fish and hardtack! Yarrrr!

AIMessageChunk(content="Arrrrg! Me favorite food be a good ol' plate of salted fish and hardtack! Yarrrr!")